## Branching Point Detector
Given a backbone model specified in the format of ```*.prototxt```, the branching point detector will automatically divide it into sequential blocks.

In [1]:
from main.auto_models import MTSeqBackbone

In [2]:
prototxt = 'models/deeplab_resnet34_adashare.prototxt' # MobileNetV2: models/mobilenetv2.prototxt
backbone = MTSeqBackbone(prototxt)
B = len(backbone.basic_blocks)
print('The number of blocks is {}.'.format(B))

The number of blocks is 17.


The user can __further specify coarse-grained branching points__ based on the auto-generated branching points by defining a mapping dictionary.
The following example maps the 17 or 32 branching points to 5 coarser ones:

| coarse |    0 |   1 |     2   |   3   |  4 |
|:------:|:------:|:---------:|:---------:|:----:|:--:|
|  fined | 0 | 1,2,3 | 4,5,6,7 | 8,9,10,11,12,13 | 14,15,16 |

__Note:__ The ```mapping``` dict has the last element ```5:[17]``` to indicate the all-shared models. In other words, if you're mapping $M$ branching points to $N$ coarser ones, ```mapping``` will contains $N+1$ elements in which the last one is ```N:[M]```.

In [3]:
coarse_B = 5
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}
# mapping = {0:[0,1,2,3,4,5,6], 1:[7,8,9,10,11,12,13,14,15,16,17], 2:[18,19,20,21,22], 
#            3:[23,24,25,26,27,28,29,30], 4:[31], 5:[32]} # mapping for MobileNetV2

## Design Spce Enumerator
Given the number of tasks $T$ and the number of branching points $B$, the design space enumerator could explore the tree-structured multi-task model architectures space completely.

In [4]:
from main.algorithms import enumerator 

In [5]:
T = 3 # NYUv2 has 3 tasks, Taskonomy has 5 tasks
layout_list = enumerator(T, coarse_B)
print('There are {} layouts in the design space.'.format(len(layout_list)))

There are 51 layouts in the design space.


## Task Accuracy Estimator
For each layout in the design space, we will estimate its task accuracy from the task accuacy of associated 2-task models (or multi-task models).

In [6]:
from main.algorithms import reorg_two_task_results, compute_weights, metric_inference

### Step 1: Load 2-task models results

__The task accuracy of all the 2-task models should be stored in excel and organized as the following example.__

* Each column represents different 2-task combinations. 
For $(a,b)-i: i \in \{0,1\}$, $(a,b)$ refers to the 2-task model of task $a$ and task $b$, and $-i$ means the current column is the accuracy of $i$-th task -- $0$ is task $a$, $1$ is task $b$.

* Each row represents the branching points of the 2-task models.
Notice that $0$ means independent models, while $B$ means all-shared models.

More examples can be found in the folder ```2task/*.xlsx```.

In [7]:
import pandas as pd
two_task_pd = pd.read_excel('2task/NYUv2_2task_metrics_resnet_1129_val_acc.xlsx',engine='openpyxl',index_col=0)
two_task_pd

,"(1, 2)-0","(1, 2)-1","(0, 2)-0","(0, 2)-1","(0, 1)-0","(0, 1)-1"
branch,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.437333,2.132027,0.800704,2.949223,2.790598,2.369396
2,-3.041407,2.524114,-0.096719,7.582977,4.868889,2.709499
3,-3.650078,3.538622,-0.789616,10.102776,-0.796604,3.726270
4,-3.293425,5.150695,-6.645483,8.176168,3.078391,5.437063
5,-3.146596,0.072837,1.548421,11.353102,5.109985,9.184597


### Step 2: Compute score weights

The 2-task results will be reorganized by ```reorg_two_task_results``` for the further computation, and the task weights for the layout scores are computed by ```compute_weights```.

In [8]:
two_task_metrics = reorg_two_task_results(two_task_pd, T, coarse_B)
score_weights = compute_weights(two_task_pd, T)

two_task_metrics
score_weights

[0.09348507957604814, 0.5780609133891011, 0.32845400703485067]

### Step 3: Compute layout score from accociated 2-task models

For each layout in the design space ```layout_list```, figure out the accociated 2-task models by ```metric_inference```, then set the final score by ```L.set_score_weighted```.

In [9]:
# Run for all L
for L in layout_list:
    print('Layout: {}'.format(L))
    
    subtree = metric_inference(L, two_task_metrics)
    print('Associated 2-task models for each task: {}'.format(subtree))     # xmc don't understand
    
    L.set_score_weighted(score_weights)
    print('Final Score: {:.4f}'.format(L.score))
    
    print('=' * 100)

Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]
Associated 2-task models for each task: {0: [[0, 1, 5], [0, 2, 5]], 1: [[1, 0, 5], [1, 2, 5]], 2: [[2, 0, 5], [2, 1, 5]]}
Final Score: 3.9328
Layout: [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]
Associated 2-task models for each task: {0: [[0, 1, 0], [0, 2, 0]], 1: [[1, 0, 0], [1, 2, 5]], 2: [[2, 0, 0], [2, 1, 5]]}
Final Score: -0.8975
Layout: [[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
Associated 2-task models for each task: {0: [[0, 1, 0], [0, 2, 0]], 1: [[1, 0, 0], [1, 2, 0]], 2: [[2, 0, 0], [2, 1, 0]]}
Final Score: 0.0000
Layout: [[{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
Associated 2-task models for each task: {0: [[0, 1, 0], [0, 2, 0]], 1: [[1, 0, 0], [1, 2, 1]], 2: [[2, 0, 0], [2, 1, 1]]}
Final Score: -0.3543
Layout: [[{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1

### Step 4: Sort the layouts

In [10]:
layout_order = sorted(range(len(layout_list)), key=lambda k: layout_list[k].score,reverse=True)

In [11]:
for i in range(0,len(layout_order)):
    print('Layout Idx: {}'.format(layout_order[i]))
    L = layout_list[layout_order[i]]
    print('Layout: {}'.format(L))
    print('Final Score: {:.4f}'.format(L.score))
    print('=' * 100)

Layout Idx: 45
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}]]
Final Score: 4.0419
Layout Idx: 0
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]
Final Score: 3.9328
Layout Idx: 50
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}]]
Final Score: 3.8196
Layout Idx: 37
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}]]
Final Score: 3.6698
Layout Idx: 49
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}]]
Final Score: 3.5462
Layout Idx: 26
Layout: [[{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}]]
Final Score: 3.0609
Layout Idx: 12
Layout: [[{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}]]
Final Score: 2.8935
Layout Idx: 46
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {1}, {0}]]
Final Score: 2.8638
Layout Idx: 48
Layout: [[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0}, {2}, {1}]

## Appendix 1: Dataloader, Loss, and Metrics

We provide dataloader, loss functions, and metrics evaluations for NYUv2 and Taskonomy.

In [12]:
from torch.utils.data import DataLoader

from data.nyuv2_dataloader_adashare import NYU_v2
from data.taskonomy_dataloader_adashare import Taskonomy
from data.pixel2pixel_loss import NYUCriterions, TaskonomyCriterions
from data.pixel2pixel_metrics import NYUMetrics, TaskonomyMetrics

In [13]:
dataroot = 'data/NYUv2' # Your root

criterionDict = {}
metricDict = {}

### NYUv2

In [14]:
tasks = ['segment_semantic','normal','depth_zbuffer']
cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 16, shuffle=True)

dataset = NYU_v2(dataroot, 'test', crop_h=321, crop_w=321)
valDataloader = DataLoader(dataset, 16, shuffle=True)

for task in tasks:
    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)

### Taskonomy

In [15]:
tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}
    
dataset = Taskonomy(dataroot, 'train', crop_h=224, crop_w=224)
trainDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

dataset = Taskonomy(dataroot, 'test_small', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for task in tasks:
    criterionDict[task] = TaskonomyCriterions(task, dataroot)
    metricDict[task] = TaskonomyMetrics(task, dataroot)

/home/tongping/mingcan/github/Pruning/TreeMTL/data/taskonomy_dataloader_adashare.py:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.mode is 'train':
/home/tongping/mingcan/github/Pruning/TreeMTL/data/taskonomy_dataloader_adashare.py:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.mode is 'train':


FileNotFoundError: [Errno 2] No such file or directory: 'data/NYUv2/taskonomy.json'

## Appendix 2: 2-Task Models & N-Task Models
We need to train all the 2-task models at different branching points for the performance table, and the n-task models we select after estimating and sorting their task accuracy. Therefore we also provide __a model generator that can automatically build up the 2-task models based on the given branching points, and the n-task models based on the given layout__.

In [ ]:
import torch
from main.auto_models import MTSeqModel
from main.algorithms import coarse_to_fined

### 2-Task Model
* Inputs: prototxt, the branching point, the number of branching points, the feature dimension and the number of class for task heads
* __Note:__
    * Given a coarse branch point, we can convert it to a fined branch point from the mapping.
    * The feature dimension can be defined by the user or derived from the backbone model.
    * Remember to select 2 tasks from the task set.

In [ ]:
coarse_branch = 3
fined_branch = mapping[coarse_branch][0]
feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

mapping[coarse_branch]
feature_dim
# backbone

512

In [ ]:
two_task = ['segment_semantic','normal'] # Select two tasks as you like
two_cls_num = {task: cls_num[task] for task in two_task}

# two_cls_num

In [ ]:
# you can choose where to branch out by leveraging the fined_branch variable
print(fined_branch)

model = MTSeqModel(prototxt, branch=fined_branch, fined_B=B, feature_dim=feature_dim, cls_num=two_cls_num)

8
Construct MTSeqModel from Layout:
[[{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}], [{0}, {1}]]


### N-Task Model
* Inputs: prototxt, a layout, the feature dimension and the number of class for task heads
* __Note:__
    * Given a layout enumerated under the coarse branching points, we can use ```coarse_to_fined``` to convert it to a layout under the fined branching points

In [ ]:
coarse_layout = layout_list[45]

# all shared multi-task model
coarse_layout = layout_list[0]
fined_layout = coarse_to_fined(coarse_layout, B, mapping)

coarse_layout

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]

In [ ]:
model = MTSeqModel(prototxt, layout=fined_layout, feature_dim=feature_dim, cls_num=cls_num)

Construct MTSeqModel from Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]


## Appendix 3: Trainer Functions

We further provide trainer functions to train the 2-task and n-task models.

In [ ]:
from main.trainer import Trainer

In [ ]:
model

MTSeqModel(
  (backbone): MTSeqBackbone(
    (inputNode): InputNode()
    (mtl_blocks): ModuleList(
      (0): ComputeBlock(
        (compute_nodes): ModuleList(
          (0): Conv2dNode(
            (basicOp): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          )
          (1): BN2dNode(
            (basicOp): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True)
          )
          (2): ReLUNode(
            (basicOp): ReLU(inplace=True)
          )
          (3): PoolNode(
            (basicOp): AbstractPool(
              (pool_op): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
            )
          )
        )
      )
      (1): ComputeBlock(
        (compute_nodes): ModuleList(
          (0): Conv2dNode(
            (basicOp): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (1): BN2dNode(
            (bas

In [17]:
trainer = Trainer(model.cuda(), tasks, trainDataloader, valDataloader, criterionDict, metricDict)
trainer.train(20000)


['segment_semantic', 'normal', 'depth_zbuffer', 'keypoints2d', 'edge_texture']